# Ejercicio 1

Codifique el algoritmo de entrenamiento para una red neuronal con RBF y pruebe su implementación en la resolución del problema `xor`, empleando los datos de la guía anterior.
Diseñe una red RBF para resolver el problema Iris (`irisbin.csv`), considerando una cantidad de parámetros similar a la red MLP de la guía anterior. 
Luego compare la velocidad de entrenamiento y el porcentaje final de clasificación obtenido con ambas arquitecturas.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def generar_particiones(filename, n, p_test):
    ds = np.genfromtxt(filename, delimiter=',')
    n_test = int(ds.shape[0] * p_test)
    n_train = ds.shape[0] - n_test
    M_test = np.zeros((n,n_test),dtype = int)
    M_train = np.zeros((n, n_train),dtype = int)

    for i in range(n):
        idx=np.random.choice(range(ds.shape[0]),ds.shape[0],replace = False)
        idx_test = idx[0:n_test]
        idx_train = idx[n_test:ds.shape[0]]
        M_test[i] = idx_test
        M_train[i] = idx_train
    
    return (ds, M_test, M_train)

In [6]:
ds, M_tst, M_trn  =  generar_particiones('gtp2datos/XOR_trn.csv', 1, 0.2)

# Generar el vector de entradas para entrenamiento
X = ds[M_trn[0,:], :-1]
# Generar el vector de salida deseada para entrenamiento
Yd = ds[M_trn[0,:], -1]

# Generar el vector de entradas para test
XTest = ds[M_tst[0,:], :-1]
# Generar el vector de salida deseada para test
YdTest = ds[M_tst[0,:], -1]

In [76]:
def kMeans(x, k):
    # k define la cantidad de grupos
    # se toman k grupos y se le asigna a cada patrón un grupo aleatorio
    group_idx = np.random.randint(0, k, x.shape[0])
    vec_reasig = np.zeros(x.shape[0])
    cant_it = 0
    means_vec = np.zeros((k, 2))

    # mientras se hayan hecho reasignaciones (sale cuando todos son verdaderos -> no hubo reasignaciones)
    while vec_reasig.all() != True:
        cant_it += 1
        
        # se calculan los centroides de cada grupo
        for i in range(k):
            group = x[group_idx == i]
            # si hay algo en el grupo
            if(group.shape[0] != 0):
                means_vec[i] =  np.mean(group, axis = 0)
            # si no hay nada en el grupo, [debe mantener el centroide anterior?]
        
        # por cada patron
        for i, pattern in enumerate(x):
            dist_vec = []
            for i_group in range(k):
                # vector distancias entre el patron y los centroides
                dist_vec.append(np.linalg.norm(pattern - means_vec[i_group]))
            # índice del centroide que tiene menor distancia
            idx_min = np.argmin(dist_vec)
            # reasignación de grupo si es necesario
            if (group_idx[i] != idx_min):
                vec_reasig[i] == False
                group_idx[i] = idx_min
            # si no hay reasignación se pone en verdadero
            else:
                vec_reasig[i] = True
            
    return group_idx, means_vec, cant_it

In [77]:
groups, means, cant_it = kMeans(X, 3)